### Step 0: Import Modules

In [2977]:
import pandas as pd
import random
import math

### Step 1: Define the Parameters

In [2978]:
# number of tables
N = 3

# Minimum physical distane required between tables
d = 2

# capacity for each table
c = [2, 2, 2]

# width for each table
w = [1, 1, 2]

# account for physical distancing margin -> add 2m to the width
for i in range(len(w)):
    w[i] += d

# height for each table
h = [1, 2, 2]

# account for physical distancing margin -> add 2m to the height
for i in range(len(h)):
    h[i] += d

# Width of dining space
Rw = 10

# Height of dining space
Rh = 10

#Parameters of used tables
tables = []

#initialize neighbouring solution
neighbouringSolution = []


establishment = [(Rw, Rh)]

### Step 2: Initialize simulated annealing variables

In [2979]:
#number of iterations that each temperature is held constant
m = 2

#initialize temperature
T = 100

#minimum amount of temperature level changes
k = 4

#temperature decrease constant
alpha = 0.2

### Step 3: Use construction heuristic to obtain initial solution

In [2980]:
#STUBBED
#INITIALIZE CURRENT INCUMBENT
d = {'ifUsed' : [0,0,0], 'x-coord' : [0,0,0], 'y-coord' : [0,0,0], 'capacity' : [2,200,4], 'table space width' : [3,100,1], 'table space height' : [3,100,1]}
df = pd.DataFrame(data=d)
df

,ifUsed,x-coord,y-coord,capacity,table space width,table space height
0,0,0,0,2,3,3
1,0,0,0,200,100,100
2,0,0,0,4,1,1


### Step 3.5: Set up helper functions to pack tables left to right

In [2981]:
#output the exact positioning of the rectangles that fit according to first fit layering procedure
def packTables(establishmentWidth, establishmentHeight, listOfTables):

    #sort list of rectangles in decreasing height,
    listOfTables.sort(reverse=True, key=getSecondElement)
    xSolution = []
    ySolution = []
    tableIndexes = []
    
    #if set is empty, return empty list
    if not listOfTables:
        return [], [], [], False
    
    minHeight = 0
    minWidth = 0
    maxHeight = establishmentWidth
    maxWidth = establishmentHeight
    
    
    for table in listOfTables:
        #if a table is smaller than the establishment
        if (table[0] < establishmentWidth and table[1] < establishmentHeight):
            #if a table can fit with all currently placed tables
            if ((table[0] <= maxWidth - minWidth) and (table[1] >= minHeight and table[1] <= maxHeight)):
                #add table to solutions
                xSolution.append(minWidth)
                ySolution.append(minHeight)
                tableIndexes.append(table[2])
                
                #set new boundaries for layer
                maxHeight = table[1]
                minWidth = table[0]
            #if a table can fit a layer above the previous
            elif(table[1] <= (establishmentHeight-maxHeight)):
                #set new boundaries for layer
                minHeight = maxHeight
                maxHeight = minHeight + table[0]
                
                #add table to solutions
                xSolution.append(minWidth)
                ySolution.append(minHeight)
                tableIndexes.append(table[2])
    
    ifFeasible = False
    if (len(listOfTables) == len(tableIndexes)):
        ifFeasible = True
    
    return xSolution, ySolution, tableIndexes, ifFeasible

def getSecondElement(item):
    return item[1]
 

### Step 4: Set up helper functions for simulated annealing algorithm

In [2982]:
def setIncumbent():
    tableConfiguration = []
    for index, row in df.iterrows():
        tableConfiguration.append(row['ifUsed'])
    return tableConfiguration
    
#adds or removes 1 random table
def tableSwap(configuration):
    modifiedConfiguration = configuration
    # generatenumber of unused tables that the removed table will be replaced by
    change = "add" if random.uniform(0, 1) > 0.5 else "remove"
    
    if (allSame( modifiedConfiguration, 0)):
        change = "add"
        
    if (allSame( modifiedConfiguration, 1)):
        return modifiedConfiguration
    
    if (change == "remove"):
        #randomly select a used table to be removed
        while (True):
            randomTableIndex = random.randint(0, len(modifiedConfiguration) - 1)
            if (modifiedConfiguration[randomTableIndex] == 1):
                modifiedConfiguration[randomTableIndex] = 0
                break;
    elif (change == "add"):
        while (True):
            randomTableIndex = random.randint(0, len(modifiedConfiguration) - 1)
            if (modifiedConfiguration[randomTableIndex] == 0):
                modifiedConfiguration[randomTableIndex] = 1
                break;

    return modifiedConfiguration

def allSame(configuration, value):
    return all(x == value for x in configuration)

#checks if a table arrangement is feasible
def isFeasible(arrangement):
    
    tempTables = getTableData(arrangement)
    
    #if a table is missing after packing, it is not feasible
    if(packTables(Rw, Rh, tempTables)[3]):
        return True
    else:
        return False

def getTableData(arrangement):
    tempTables = []
    for index, row in df.iterrows():
        if(arrangement[index]==1):
            tempTables.append((row['table space width'], row['table space height'], index))
    return tempTables
    

#Generate feasible neighbouring solution by packing a subset of all available tables
def generateFeasibleSolution(currentSolution):
    print ("generating neighbour of:", currentSolution)
    randomSolution = []
    
    solutionFound = False
    while(not solutionFound):
        randomSolution = tableSwap(currentSolution[:])
        if(isFeasible(randomSolution)):
            solutionFound = True    

    return randomSolution

#Check how a feasible solution impacts the objective
def checkObjective(configuration):
    objective = 0
    index = 0
    for table in configuration:
        if configuration[index]==1:
            objective += df.at[index,'capacity']
        index += 1
    return (objective)

#Intitialize temperature schedule
def initializeTemperatureSchedule(T, alpha, k):
    latestTemp = T
    temperatureSchedule = [T]
    for i in range(k-1):
        temperatureSchedule.append(latestTemp*alpha)
        latestTemp = latestTemp*alpha
    return temperatureSchedule
    
    

### Step 4: Execute simulated annealing algorithm

In [2983]:
def acceptCandidate(currentSolutionObjective, candidateSolutionObjective, T):
    #generate random number between 0 and 1
    randomNumber = random.uniform(0, 1)
    acceptanceProbability = math.exp((candidateSolutionObjective - currentSolutionObjective)/T)
    
    #if candidate solution is better, then accept it
    if (candidateSolutionObjective > currentSolutionObjective):
        return 'betterCandidateAccepted'
    elif (randomNumber <= acceptanceProbability):
        return 'worseCandidateAccepted'
    else:
        return 'worseCandidateRejected'

def executeAnnealingAlgorithm():
    
    #construct initial incumbent
    incumbent = setIncumbent()
    incumbentObjective = checkObjective(incumbent)
        
    #initialize current solution to be the incumbent
    currentSolution = incumbent
    currentSolutionObjective = checkObjective(incumbent)

    #initializeTemperatureSchedule
    temperatureSchedule =  initializeTemperatureSchedule(T, alpha, k)
    print('Temperature schedule: ', temperatureSchedule)
    

    for temperature in temperatureSchedule:
        for index in range(m):
            print ('Temperature: ', temperature)
            print("INCUMBENT: ", incumbent)
            print("INCUMBENT OBJECTIVE: ", incumbentObjective)
            print('CURRENT SOLUTION: ',currentSolution)
            print('CURRENT SOLUTION OBJECTIVE: ',currentSolutionObjective)
            
            #generate a candidate solution
            candidateSolution = generateFeasibleSolution(currentSolution)
            candidateSolutionObjective = checkObjective(candidateSolution)
            
            #display candidate data
            displaySolutionData(candidateSolution, candidateSolutionObjective, False)
    
            #accept candidate solution to be current solution with some degree of probability
            acceptanceResult = acceptCandidate(currentSolutionObjective, candidateSolutionObjective, temperature)
            print(acceptanceResult, '\n')
            
            #adjust current solution if candidate is better or if adjust current solution if it is worse with some probability
            if (acceptanceResult == 'betterCandidateAccepted' or acceptanceResult == 'worseCandidateAccepted' ):
                currentSolution = candidateSolution
                currentSolutionObjective = candidateSolutionObjective

            #only replace incumbent if the candidate is better than it
            if(currentSolutionObjective > incumbentObjective):
                print("INCUMBENT OBJ CHANGE FROM:", incumbentObjective, "to", currentSolutionObjective)
                print("INCUMBENT CHANGE FROM:", incumbent, "to", currentSolution)
                incumbent = currentSolution
                incumbentObjective = currentSolutionObjective
    
    displaySolutionData(incumbent, incumbentObjective, True)

def displaySolutionData(solution, solutionObjective, ifIncumbent):
    
    if (ifIncumbent):
        print('Final Incumbent Solution: ', solution)
        print('Final Incumbent Objective Solution: ', solutionObjective)
        print('Final Incumbent x Values: ', packTables(Rw, Rh, getTableData(solution))[0])
        print('Final Incumbent y Values: ', packTables(Rw, Rh, getTableData(solution))[1])
    else:
        print('Neighbouring Solution: ',solution)
        print('Neighbouring Solution Objective: ', solutionObjective)
        print('neighbouring Solution x Values: ', packTables(Rw, Rh, getTableData(solution))[0])
        print('neighbouring Solution y Values: ', packTables(Rw, Rh, getTableData(solution))[1])
    

executeAnnealingAlgorithm()


Temperature schedule:  [100, 20.0, 4.0, 0.8]
Temperature:  100
INCUMBENT:  [0, 0, 0]
INCUMBENT OBJECTIVE:  0
CURRENT SOLUTION:  [0, 0, 0]
CURRENT SOLUTION OBJECTIVE:  0
generating neighbour of: [0, 0, 0]
Neighbouring Solution:  [0, 0, 1]
Neighbouring Solution Objective:  4
neighbouring Solution x Values:  [0]
neighbouring Solution y Values:  [0]
betterCandidateAccepted 

INCUMBENT OBJ CHANGE FROM: 0 to 4
INCUMBENT CHANGE FROM: [0, 0, 0] to [0, 0, 1]
Temperature:  100
INCUMBENT:  [0, 0, 1]
INCUMBENT OBJECTIVE:  4
CURRENT SOLUTION:  [0, 0, 1]
CURRENT SOLUTION OBJECTIVE:  4
generating neighbour of: [0, 0, 1]
Neighbouring Solution:  [1, 0, 1]
Neighbouring Solution Objective:  6
neighbouring Solution x Values:  [0, 3]
neighbouring Solution y Values:  [0, 0]
betterCandidateAccepted 

INCUMBENT OBJ CHANGE FROM: 4 to 6
INCUMBENT CHANGE FROM: [0, 0, 1] to [1, 0, 1]
Temperature:  20.0
INCUMBENT:  [1, 0, 1]
INCUMBENT OBJECTIVE:  6
CURRENT SOLUTION:  [1, 0, 1]
CURRENT SOLUTION OBJECTIVE:  6
genera